In [ ]:
import datetime as dt
import sys
import os

from dotenv import load_dotenv, find_dotenv

sys.path.append("../")
import woolworther as ww

load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2


In [ ]:
# Read a watchlist

stock_codes = ww.read_watchlist(ww.DATA_DIR / "watchlist.csv")
display(stock_codes)

watchlist_url = "https://docs.google.com/spreadsheets/d/1Cu8wEckdfGPtkN9zj5H8O_4ZUrQyrXBZ8Um-K2BSlnc/edit?usp=drive_link"
stock_codes = ww.read_watchlist(watchlist_url)
display(stock_codes)


In [ ]:
# Get and parse a product

r = ww.get_product(stock_codes[0])
ww.parse_product(r)

In [ ]:
# Get and parse many products

%time f = ww.collect_products(stock_codes)
f

In [ ]:
# Filter sales

ww.filter_sales(f)

In [ ]:
# Get products in bulk

ww.run_pipeline(watchlist_url)

In [ ]:
# Email sales
recipients = ["ibatatas@runbox.com"]
print(f"Emailing {recipients}")

username = os.getenv('GMAIL_USERNAME')  # Replace with your GMail username
password = os.getenv('GMAIL_PASSWORD')  # Replace with your GMail password
ww.run_pipeline(
    watchlist_url, 
    recipients, 
    gmail_username=username, 
    gmail_password=password, 
    headers={"From": "Woolworther"},
    sales_only=True,
)